## Árvores de regressão - exercícios 02

Este exercício será uma continuação do anterior, mesma base, mesmas variáveis - vamos tentar buscar a 'melhor árvore'.

A descrição das variáveis está abaixo:

| Variavel | Descrição|
|-|-|
|CRIM| taxa de crimes per-cápita da cidade | 
|ZN| proporção de terrenos residenciais zoneados para lotes com mais de 25.000 pés quadrados. |
|INDUS| proporção de acres de negócios não varejistas por cidade |
|CHAS |vale 1 se o terreno faz fronteira com o *Charles River*; 0 caso contrário |
|NOX| Concentração de óxido nítrico (em partes por 10 milhões) |
|RM| número médio de quartos por habitação |
|AGE| proporção de unidades ocupadas pelo proprietário construídas antes de 1940 |
|DIS| distâncias ponderadas até cinco centros de empregos de Boston |
|RAD| índice de acessibilidade a rodovias radiais |
|TAX| taxa de imposto sobre a propriedade de valor total por \\$10,000 |
|PTRATIO| razão pupilo-professor da cidade |
|B| $ 1000 (Bk - 0,63) ^ 2 $ onde Bk é a proporção de negros por cidade |
|LSTAT| \%status inferior da população |
|MEDV| (variável resposta) Valor mediano das casas ocupadas pelo proprietário em US $ 1.000|

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import datasets  
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn import tree

%matplotlib inline

In [ ]:
data_url = 'https://raw.githubusercontent.com/rhatiro/Curso_EBAC-Profissao_Cientista_de_Dados/main/Mo%CC%81dulo%2011%20-%20A%CC%81rvores%20II%20(Parte%20I-%20a%CC%81rvore%20de%20regressa%CC%83o)/database/boston_clean_data.csv'
df = pd.read_csv(filepath_or_buffer=data_url)

X = df.drop(columns='MEDV')
y = df['MEDV']

print('Quantidade de linhas e colunas de X:', X.shape)
print('Quantidade de linhas de y:', len(y), '\n')

print(df.info())
df

In [ ]:
X.head()

In [ ]:
y.head()

### 1. Execute os passos do exercício anterior, até que você tenha uma árvore de regressão predizendo o valor do imóvel na base de treinamento.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2402)

dt_reg = DecisionTreeRegressor(max_depth=8, random_state=2402)
dt_reg.fit(X_train, y_train)

In [ ]:
print('Erro Quadrático Médio (MSE):', 
      round(mean_squared_error(y_true=y_train, 
                               y_pred=dt_reg.predict(X_train)), 
            2))

print('Coeficiente de determinação (R-quadrado):', 
      round(dt_reg.score(X=X_train, 
                         y=y_train), 
            2))

In [ ]:
pd.concat(objs=[y_train.reset_index(drop=True), 
                pd.DataFrame(data=dt_reg.predict(X_train), 
                             columns=['Valor predito do imóvel na base de treinamento']
                            )
               ], axis=1)

### 2.  Calcule o caminho indicado pelos CCP-alfas dessa árvore.

In [ ]:
path = dt_reg.cost_complexity_pruning_path(X=X_train, y=y_train)
path

In [ ]:
ccp_alphas, impurities = path.ccp_alphas, path.impurities

plt.figure(figsize=(12,6))
plt.plot(ccp_alphas, impurities)

plt.xlabel(xlabel='Alpha efetivo')
plt.ylabel(ylabel='Impureza total das folhas')

plt.show()

### 3. Paca cada valor de alpha obtido no item 2, treine uma árvore com o respectivo alfa, e guarde essa árvore em uma lista.

In [ ]:
clfs = []

for ccp_alpha in ccp_alphas:
    clf = DecisionTreeRegressor(ccp_alpha=ccp_alpha, random_state=2402)
    clf.fit(X_train, y_train)
    clfs.append(clf)

In [ ]:
tree_depths = [clf.tree_.max_depth for clf in clfs]

plt.figure(figsize=(12, 6))
plt.plot(ccp_alphas, tree_depths)

plt.xlabel(xlabel='Effective alpha')
plt.ylabel(ylabel='Tree depth')

plt.show()

### 4. Para cada árvore na lista, calcule o MSE da árvore.

In [ ]:
train_scores = [mean_squared_error(y_true=y_train, 
                                   y_pred=clf.predict(X_train)) for clf in clfs]

test_scores  = [mean_squared_error(y_true=y_test, 
                                   y_pred=clf.predict(X_test )) for clf in clfs]

### 5. Monte um gráfico do MSE pelo alpha, escolha um valor de alpha perto do ponto de mínimo do MSE

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))

ax.set_xlabel(xlabel='alpha')
ax.set_ylabel(ylabel='MSE')
ax.set_title(label='MSE  X  Alpha do conjunto de dados de treino e teste')

ax.plot(ccp_alphas[:-6], train_scores[:-6], 
        marker='o', label='treino', drawstyle='steps-post')

ax.plot(ccp_alphas[:-6], test_scores[:-6], 
        marker='o', label='teste', drawstyle='steps-post')

ax.legend()

plt.show()


In [ ]:
df_ccp = pd.DataFrame(data={'ccp_alphas':ccp_alphas, 
                            'train_scores':train_scores, 
                            'test_scores':test_scores})

df_ccp.sort_values(by=['test_scores', 'train_scores'])

In [ ]:
ccp_alpha_min = df_ccp.sort_values(by=['test_scores', 'train_scores']).iloc[0,0]
ccp_alpha_min

### 6. Calcule o R-quadrado dessa árvore encontrada no item acima

In [ ]:
final_tree = DecisionTreeRegressor(ccp_alpha=ccp_alpha_min, random_state=2402)
final_tree.fit(X_train, y_train)

print(f'Profundidade: {final_tree.tree_.max_depth}')
print(f'R-quadrado na base de testes: {final_tree.score(X_test, y_test):.2f}')
print(f'MSE na base de testes: {mean_squared_error(y_test, final_tree.predict(X_test)):.2f}')

### 7. Visualize esta árvore.

In [ ]:
plt.rc('figure', figsize=(12,6))

tree.plot_tree(decision_tree=final_tree, 
               feature_names=X.columns, 
               filled=True);